Import libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn
import pickle
import os
 
from sklearn.model_selection import cross_validate, ParameterGrid, train_test_split, KFold
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

from category_encoders.cat_boost import CatBoostEncoder

from xgboost import XGBRegressor

sklearn.set_config(transform_output = "pandas")

In [2]:
data = pd.read_csv('datasets/clean/data.csv')
data.head()

,Age,Brands,Cilinder,Color,Currency,Engine,Fuel_type,KM,Location,Models,Price,Transmission,Upholstery,Urlpic,Version,Year
0,1.0,SUZUKI,3.0,rojo,USD,1000.0,gasolina,16684.0,Lima,S-PRESSO,9800.0,mecanica,tela,https://cde.neoauto.pe/autos_usados/360x240/66...,4x2,2023.0
1,1.0,DFSK,4.0,blanco,USD,1500.0,gas_glp,11054.0,Lima,GLORY 500,12000.0,mecanica,tela,https://cde.neoauto.pe/autos_usados/360x240/70...,delantera,2023.0
2,1.0,TOYOTA,4.0,rojo,USD,1300.0,gas_gnv,15500.0,Lima,YARIS,15500.0,mecanica,tela,https://cde.neoauto.pe/autos_usados/360x240/70...,delantera,2023.0
3,4.0,FORD,6.0,blanco,USD,3500.0,gasolina,12500.0,Lima,RAPTOR,75900.0,automatica_secuencial,tela,https://cde.neoauto.pe/autos_usados/360x240/30...,4x4,2020.0
4,1.0,KIA,4.0,gris,USD,1999.0,gasolina,52130.0,Lima,SPORTAGE,27900.0,mecanica,cuero,https://cde.neoauto.pe/autos_usados/360x240/70...,4x2,2023.0


Clean data

In [3]:
cols_to_remove = ['Urlpic', 'Year', 'Currency']
data.drop(columns = cols_to_remove, inplace = True)
data_clean = data.dropna(subset = ['Price'])

**Find best model**

In [4]:
# joinn all csv files
results = pd.concat([pd.read_csv(f'tuning/{csv_file}') for csv_file in sorted(os.listdir('tuning'))], axis = 0, ignore_index = True)

# filter results according to train > valid & train > test
results = results[(results['opt_train'] > results['opt_val']) & (results['opt_train'] > results['opt_test'])].reset_index(drop = True)

# select best iteration
best_iter = results.loc[results['opt_test'].idxmax(), 'iter']

# choose param tuned
param_tuned = results[results['iter'] == best_iter].reset_index(drop = True)

# create overfitting flag
results['valid_overfitting'] = np.where(results['opt_overfitting'] < 0.1, 1, 0)
var_left = results[results['valid_overfitting'] == 1].reset_index(drop = True)
best = var_left.groupby('iter', as_index = False)\
               .agg({'opt_test': ['mean', 'min', 'max', 'std'],
                     'valid_overfitting': ['count'],
                     'opt_overfitting': ['mean']})

best.columns = ['iter', 'mean', 'min', 'max', 'std', 'n_valid', 'opt_overfitting']
best['cv'] = best['std'] / best['mean']
for agg_func in ['mean', 'max', 'opt_overfitting', 'n_valid', 'cv']:
    best[f'{agg_func}_rank'] = pd.qcut(best[agg_func], 10, labels = False, duplicates = 'drop')

best['points'] = best[[col for col in best.columns if 'rank' in col]].sum(axis = 1)
best.sort_values(by = ['points', 'mean'], ascending = [False, False], ignore_index = True, inplace = True)

# get param tuned
param_tuned = var_left[var_left['iter'] == best['iter'].tolist()[0]].reset_index(drop = True)

# get best seed
best_seed = param_tuned.loc[param_tuned['seed'].idxmax(), 'seed']

# get best hyperparameters
best_params = param_tuned[param_tuned['seed'] == best_seed].sort_values(by = ['best_overfitting'], ignore_index = True)
best_params.head(1)

,iter,seed,cv_fold,params,n_trees,opt_n_trees,opt_train,opt_val,opt_test,opt_overfitting,best_n_trees,best_train,best_val,best_test,best_overfitting,valid_overfitting
0,0,99,0,"{'alpha': 1, 'gamma': 0.1, 'learning_rate': 0....",16,15,0.690283,0.679005,0.661875,0.028408,15,0.690283,0.679005,0.661875,0.028408,1


**Refit model with found hyperparameters**

In [5]:
# get best record
record = best_params.to_dict('records')[0]

# split data into subsets
trainval, test = train_test_split(data_clean, train_size = 0.8, shuffle = True, random_state = record['seed'])

# get X and y matrices
X = trainval.drop(columns = 'Price').copy()
y = trainval['Price'].copy()

X_test = test.drop(columns = 'Price').copy()
y_test = test['Price'].copy()

# get subsets with kfold
cv = KFold(n_splits = 5, shuffle = True, random_state = record['seed'])

train_index, val_index = list(cv.split(X, y))[0]

# get train and val set
X_train, y_train = X.iloc[train_index], y.iloc[train_index]
X_val, y_val = X.iloc[val_index], y.iloc[val_index]

In [12]:
# declare imputer and transformer
encoder = CatBoostEncoder(handle_missing = 'return_nan',
                        handle_unknown = 'return_nan')

# declare transformer
transformer = ColumnTransformer([('enc', encoder, make_column_selector(dtype_include = ['object', 'category']))],
                                remainder = 'passthrough',
                                verbose_feature_names_out = False)

# declare estimator
estimator = XGBRegressor(objective = 'reg:squarederror',
                         eval_metric = r2_score,
                         n_estimators = record['n_trees'],
                         **eval(record['params']))

# declare pipeline
preprocessing = Pipeline([('enc', encoder), ('trs', transformer)])

# fit preprocessing
X_train = preprocessing.fit_transform(X_train, y_train)
X_val = preprocessing.transform(X_val)
X_test = preprocessing.transform(X_test)

# fit estimator
estimator.fit(X = X_train,
              y = y_train,
              eval_set = [(X_train, y_train),
                          (X_val, y_val),
                          (X_test, y_test)],
              verbose = 0)

# save objects
objects = [('encoder', encoder),
           ('transformer', transformer),
           ('preprocessing', preprocessing),
           ('estimator', estimator)]

for obj_name, obj in objects:
    with open(f'objects/{obj_name}.pkl', 'wb') as file:
        pickle.dump(obj, file)

**Reevaluate model**

In [13]:
# get best record
record = best_params.to_dict('records')[0]

# split data into subsets
trainval, test = train_test_split(data_clean, train_size = 0.8, shuffle = True, random_state = record['seed'])

# get X and y matrices
X = trainval.drop(columns = 'Price').copy()
y = trainval['Price'].copy()

X_test = test.drop(columns = 'Price').copy()
y_test = test['Price'].copy()

# get subsets with kfold
cv = KFold(n_splits = 5, shuffle = True, random_state = record['seed'])

train_index, val_index = list(cv.split(X, y))[record['cv_fold']]

# get train and val set
X_train, y_train = X.iloc[train_index], y.iloc[train_index]
X_val, y_val = X.iloc[val_index], y.iloc[val_index]

In [15]:
# read objects
with open(f'objects/encoder.pkl', 'rb') as file:
    encoder = pickle.load(file)

with open(f'objects/transformer.pkl', 'rb') as file:
    transformer = pickle.load(file)

with open(f'objects/preprocessing.pkl', 'rb') as file:
    preprocessing = pickle.load(file)

with open(f'objects/estimator.pkl', 'rb') as file:
    estimator = pickle.load(file)

# create pipeline
pipe = Pipeline([('prep', preprocessing), ('est', estimator)])

# merge datasets
data_train = pd.concat([X_train, y_train], axis = 1).assign(sample = 'train')
data_val = pd.concat([X_val, y_val], axis = 1).assign(sample = 'val')
data_test = pd.concat([X_test, y_test], axis = 1).assign(sample = 'test')

data_prep = pd.concat([data_train, data_val, data_test], axis = 0, ignore_index = True)

# create punctuations
data_prep['Price_predict'] = pipe.predict(data_prep[estimator.get_booster().feature_names])

# get first metrics
summary = data_prep.groupby('sample')\
                   .apply(lambda df: r2_score(df['Price'], df['Price_predict']), include_groups = False)\
                   .loc[['train', 'val', 'test']]
summary

sample
train    0.689514
val      0.679005
test     0.661875
dtype: float64

See results

In [16]:
data_prep.describe()

,Age,Cilinder,Engine,KM,Price,Price_predict
count,6903.000000,5869.000000,6.402000e+03,6.740000e+03,6904.000000,6904.000000
mean,7.680719,4.315727,4.542234e+03,6.942658e+04,19229.570158,19307.250000
std,6.600145,1.001359,1.202761e+05,1.127016e+05,14396.858724,8900.935547
min,0.000000,1.000000,1.000000e+00,1.000000e+01,1.000000,7579.339355
25%,3.000000,4.000000,1.591000e+03,3.000000e+04,11000.000000,13523.563965
50%,6.000000,4.000000,1.998000e+03,5.650000e+04,16000.000000,16987.833984
75%,10.000000,4.000000,2.400000e+03,9.000000e+04,23000.000000,22214.994141
max,97.000000,14.000000,8.816657e+06,7.824000e+06,310000.000000,86579.796875
